## Classification + Regression - ResNet50 

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# setup CUDA_VISIBLE DEVICES for titan.sci.utah.edu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [ ]:
# This file contains all the main external libs we'll use

from fastai.imports import *
from fastai.conv_learner import *

from fastai.plots import *


PATH = "../data/"
sz=224
arch=resnet50
bs=8

In [ ]:
# CSV for classification task
Classification_csv = f'{PATH}Dataset.csv'


In [ ]:
Classification_df = pd.read_csv(Classification_csv)
Classification_df.head()

In [ ]:
# CSV for regresssion task
Regression_csv = f'{PATH}Dataset_Regression.csv'

In [ ]:
Regression_df = pd.read_csv(Regression_csv)
Regression_df.head()

In [ ]:
# Validation indices

n = len(list(open(Regression_csv)))-1
# Return validation indexes using a 10% split
val_idxs = get_cv_idxs(n,val_pct=0.1)
print('n:',n)
print('Nb val_idxs',len(val_idxs))
print('val_idxs',val_idxs)

In [ ]:
# Transformations
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on)


In [ ]:
# Define regression dataset
md = ImageClassifierData.from_csv(PATH, 'data_all', Regression_csv, tfms=tfms, continuous=True, 
   bs=bs, val_idxs=val_idxs, num_workers = 0)

In [ ]:
# Define classification dataset
md2 = ImageClassifierData.from_csv(PATH, 'data_all', Classification_csv, tfms=tfms, continuous=False, 
   bs=bs, val_idxs=val_idxs, num_workers = 0)

In [ ]:
class ConcatLblDataset(Dataset):
    def __init__(self, ds, y2): self.ds,self.y2 = ds,y2
    def __len__(self): return len(self.ds)
    
    def __getitem__(self, i):
        x,y = self.ds[i]
        return (x, (y,self.y2[i]))

In [ ]:
# Combine datasets
trn_ds2 = ConcatLblDataset(md.trn_ds, md2.trn_y)
val_ds2 = ConcatLblDataset(md.val_ds, md2.val_y)

In [ ]:
md.trn_dl.dataset = trn_ds2
md.val_dl.dataset = val_ds2

In [ ]:
trn_ds2[150][1]


In [ ]:
val_ds2[20][1]

In [ ]:
x,y=next(iter(md.val_dl))
idx=3
ima=md.val_ds.ds.denorm(to_np(x))[idx]
to_np(y[0][idx])

In [ ]:
y[1][idx]

In [ ]:
y[0]

In [ ]:
y[1]

In [ ]:
y[0][0]

In [ ]:
y[1][0]

In [ ]:
plt.imshow(ima)

In [ ]:
# Add simple dense model (instead of default fastai model)
#head_reg = nn.Sequential(Flatten(), nn.Linear(100352,1))
#learn = ConvLearner.pretrained(arch, data, xtra_fc=[], ps=0.25, precompute=False)

#learn = ConvLearner.pretrained(arch, data, precompute=False)

In [ ]:
head_reg = nn.Sequential(
    AdaptiveConcatPool2d(),
    Flatten(),
    nn.BatchNorm1d(4096),
    nn.Dropout(0.25),
    nn.Linear(4096,1+5),
)

models = ConvnetBuilder(arch, 0, 0, 0, custom_head=head_reg)

learn = ConvLearner(md, models)
learn.opt_fn = optim.Adam

In [ ]:
learn

In [ ]:
# Cell to run for this notebook
# Adaptation from "pascal_Clem" notebook
def detn_loss(input, target):
    r_t,c_t = target
    r_i,c_i = input[:, :1], input[:, 1:]
    r_i = F.sigmoid(r_i)*200+600
    # I looked at these quantities separately first then picked a multiplier
    #   to make them approximately equal
    #print('r_t',r_t)
    #print('c_t',c_t)
    #print('r_i',r_i)
    #print('c_i',c_i)
    #print('\nL1_loss',F.l1_loss(r_i, r_t))
    #print('Cross_entropy loss',F.cross_entropy(c_i, c_t))
    return F.l1_loss(r_i, r_t) + (F.cross_entropy(c_i, c_t)*20)

def detn_l1(input, target):
    r_t,_ = target
    r_i = input[:, :1]
    r_i = F.sigmoid(r_i)*200+600
    #print('r_i',r_i)
    #print('r_t',r_t)
    return F.l1_loss(V(r_i),V(r_t)).data


def detn_acc(input, target):
    _,c_t = target
    c_i = input[:, 1:]
    #print('c_i',V(c_i))
    #print('c_t',V(c_t))
    #return V(C_i)
    # Code from Clem
    #preds = np.argmax(V(c_i), 1)
    return accuracy_np(to_np(c_i), to_np(c_t))
    #return (preds==V(c_t)).mean()

learn.crit = detn_loss
learn.metrics = [detn_l1, detn_acc]

In [ ]:
lrf=learn.lr_find(end_lr=10)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot(n_skip = 5)

In [ ]:
#learn.fit(5e-3, 5)
#learn.precompute=False

In [ ]:
learn.fit(5e-3, 5)

In [ ]:
learn.fit(5e-3, 5, cycle_len = 1)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_lastlayer_ResNet50_ClassificationAndRegression')

In [ ]:
learn.load('224_lastlayer_ResNet50_ClassificationAndRegression')

In [ ]:
learn.unfreeze()
lr = 5e-3
lrs=np.array([lr/9,lr/3,lr])

In [ ]:
lrf=learn.lr_find()

In [ ]:
learn.sched.plot(n_skip=5)

In [ ]:
lr = 5e-4
lrs=np.array([lr/9,lr/3,lr])
learn.fit(lrs, 5, cycle_len = 1, cycle_mult = 2)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_all_ResNet50_ClassificationAndRegression')

In [ ]:
learn.load('224_all_ResNet50_ClassificationAndRegression')

In [ ]:
preds = learn.predict()
#print(preds)
print(preds)

#log_preds,y = learn.predict()
#probs = np.mean(preds,0)
#print(probs)
#accuracy_np(probs,y)

In [ ]:
md.val_ds[0][1]

## Analyzing results

In [ ]:
from sklearn.metrics import *
import math
MAE = mean_absolute_error(y, probs)
RMSE = math.sqrt(mean_squared_error(y, probs))
print('MAE', MAE)
print('RMSE', RMSE)

In [ ]:
# R2 calculation
y_mean = np.mean(y, 0)
print('y_mean', y_mean)

y_mean_array = np.full(len(y),y_mean)
#print('y_mean_array',y_mean_array)

SStot = np.sum((y-y_mean)**2)
SSres = np.sum((y-probs)**2)
R2 = 1 - (SSres/SStot)
print('SStot', SStot)
print('SSres', SSres)
print('R2', R2)